<a href="https://colab.research.google.com/github/samiha-mahin/A-Machine-Learning-Models-Repo/blob/main/Bagging_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### 🌳 What is Bagging?

**Bagging** stands for **Bootstrap Aggregating**.

It’s an **ensemble learning technique** that:

* Combines the predictions of **multiple models** (usually of the same type).
* Aims to **reduce variance** and **prevent overfitting**.
* Works especially well with **unstable models** (like decision trees).

---

### 🧠 How Does It Work?

1. **Bootstrapping (Sampling with replacement):**

   * From the original dataset, you create **multiple new datasets** by randomly selecting data points **with replacement**.
   * Each dataset is slightly different.

2. **Train models:**

   * You train a **separate model** on each of these datasets.
   * Typically, the base model is a **Decision Tree**.

3. **Aggregate the predictions:**

   * For **classification**: Take the **majority vote**.
   * For **regression**: Take the **average** of all outputs.

---

### 🍎 Real-Life Analogy

Imagine you're trying to decide whether a new fruit is an **apple or a tomato**.

* You ask **10 friends**, but give each of them a slightly different photo of the fruit (some brighter, some darker).
* Each friend gives you their answer.
* You **take the majority vote** to decide what the fruit is.

→ This is **bagging** in action — using **multiple weak opinions** to make a **strong, stable decision**.

---

### 📊 Proper Example: Predicting Loan Approval

Suppose you're building a model to predict if a person should get a loan.

#### Without Bagging:

* You train **one decision tree**.
* If the data changes a little, the tree might give **completely different results** (high variance).

#### With Bagging:

* You create 10 datasets from the original one (bootstrapped).
* Train 10 decision trees (one on each dataset).
* For a new applicant:

  * Each tree makes a prediction (Yes or No).
  * You take the **majority vote**.

✅ Result:

* **More stable**
* **Less likely to overfit**
* **Higher accuracy**

---

### 📦 Popular Bagging Example: **Random Forest**

* Random Forest = Bagging of **Decision Trees**
* It’s one of the most powerful and widely used ML algorithms.

---

### ✅ Benefits of Bagging

| Feature            | Benefit                                |
| ------------------ | -------------------------------------- |
| Reduces Variance   | Makes unstable models more stable      |
| Avoids Overfitting | Prevents one model from learning noise |
| Easy to Implement  | Especially with decision trees         |
| Improves Accuracy  | Better than a single model             |



In [ ]:
import pandas as pd

df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.Outcome.value_counts()

,count
Outcome,
0,500
1,268


In [ ]:
X = df.drop('Outcome', axis='columns')
y = df.Outcome

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3] #first 3 rows of the scaled feature data

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)


In [16]:
X_train.shape

(576, 8)

In [14]:
X_test.shape

(192, 8)

In [17]:
y_train.value_counts()

,count
Outcome,
0,375
1,201


In [18]:
y_test.value_counts()

,count
Outcome,
0,125
1,67


# **Train using Bagging**

In [22]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_model = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 100 ,
    max_samples= 0.8,
    oob_score= True,
    random_state= 0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7534722222222222

🔍 Parameter Breakdown:

| Parameter                                 | Meaning                                                                                               |
| ----------------------------------------- | ----------------------------------------------------------------------------------------------------- |
| `estimator=DecisionTreeClassifier()`    | Each individual model is a decision tree.                                                             |
| `n_estimators=100`                        | Use 100 different decision trees in the ensemble.                                                     |
| `max_samples=0.8`                         | Each tree is trained on a **random 80% sample** (with replacement) from the training data.            |
| `oob_score=True`                          | Use the **out-of-bag** samples to estimate the model's accuracy **without needing a validation set**. |
| `random_state=0`                          | Ensures **reproducibility** (same result every time).                                                 |


In [23]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [26]:
from sklearn.model_selection import cross_val_score
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

In [27]:
scores.mean()

np.float64(0.7578728461081402)